# TIES5830 Assignment 4

#### Author: Heikki Kurhinen

### Task 1:

Implement probability of selection for single objective optimization. * (refer eqn.
12, 13, 14)

In [5]:
import numpy as np

# Calculates probability of selection
def probability_of_selection(x, oa, ob):
    A = x[0]
    B = x[1]
    m = (A - B) / ob
    s = oa / ob
    return 1 / (1 + np.exp(-1 * ((2.5*m)/np.sqrt(2+2*s**2))))

### Task 2:

- Implement probability of dominance and ranking. * (refer eqn. 15, 17, 18, 22)
 - Generate 50 samples from DTLZ2 problem (3 objectives, 5 decision variables) using LHS sampling
 - Build Kriging surrogates using the generated samples
 - Resample or generate 20 data points randomly
 - Rank the 20 points using probability of dominance ranking

In [12]:
# Calculates probability of dominance
# returns both P(A>B) and P(A<B)
def probability_of_dominance(A, B, sigma):
    pos = lambda x : probability_of_selection(x, sigma, sigma)
    pairs = np.column_stack((A, B))
    aqtb = np.array(list(map(pos, pairs)))
    bqta = np.array(list(map(lambda p: 1 - p, aqtb)))
    return np.prod(aqtb), np.prod(bqta)

# Helper to calculate weak dominance
def weak_dominance(aqtb, bqta):
    return 1 - bqta - aqtb 

# Calculates probabilistic ranking for individuals
def probabilistic_ranking(x, xs, sigma):
    pds = []
    wds = []
    for z in xs:
        aqtb, bqta = probability_of_dominance(x, z, sigma)
        pds.append(aqtb)
        wds.append(weak_dominance(aqtb, bqta))
    return np.sum(pds) + 0.5*np.sum(wds) - 0.5

# Test data taken from the research paper to verify correct results
ns = [
  [2, 8],
  [3, 5],
  [5, 3],
  [10, 2],
  [10.5, 5],
  [11, 8.5]
]

for n in ns:
    print(probabilistic_ranking(n, ns, 0.1), probabilistic_ranking(n, ns, 1))

2.000965230651306 2.2729855480934447
1.7499981366803579 1.6450100785787027
1.4999981366873019 1.4183256906614985
1.500967093970948 1.9193591599019593
3.249999999993056 3.217716064256361
4.99807140201703 4.526603458508033


In [8]:
from desdeo_problem.testproblems import test_problem_builder
from scipy.stats import qmc
from sklearn.gaussian_process import GaussianProcessRegressor

# Method for generating samples using LHS-sampling
def lhs(count, d):
    sampler = qmc.LatinHypercube(d=d)
    return sampler.random(n=count)

# Test problem (DTLZ2 with 3 objectives and 5 decision variables)
obj = test_problem_builder("DTLZ2", 5, 3)

# 50 random samples for creating training data
training_data = lhs(50, 5)
target_values = obj.evaluate(training_data).objectives
objective_1_values = target_values[:,0]
objective_2_values = target_values[:,1]
objective_3_values = target_values[:,2]

# Kriging surrogates
gprO1 = GaussianProcessRegressor()
gprO2 = GaussianProcessRegressor()
gprO3 = GaussianProcessRegressor()
gprO1.fit(training_data, objective_1_values)
gprO2.fit(training_data, objective_2_values)
gprO3.fit(training_data, objective_3_values)

# Test data and results from surrogates
test_data = lhs(20, 5)
gpO1Values = gprO1.predict(test_data)
gpO2Values = gprO2.predict(test_data)
gpO3Values = gprO3.predict(test_data)

solutions = np.column_stack((gpO1Values, gpO2Values, gpO3Values))

In [11]:
# Objective values and their rankings
for s in solutions:
    print(f"f1={s[0]}, f2={s[1]}, f3={s[2]}, ranking={probabilistic_ranking(s, solutions, 0.1)}")

f1=0.038082020966360375, f2=0.11687535321636844, f3=1.110699633928407, ranking=8.403859722088646
f1=0.11967176796299839, f2=0.05824926887494897, f3=1.4107411905811977, ranking=9.621400686535743
f1=0.46865688994577326, f2=1.1776632255054302, f3=0.5056845387661371, ranking=9.697674322817237
f1=0.3759697093636376, f2=0.34406914921185283, f3=1.0187022993395374, ranking=9.541091572793949
f1=0.48730584806711974, f2=1.1743213068507803, f3=0.6985847253024859, ranking=10.00350272411082
f1=0.1894746350277252, f2=0.3168890613442148, f3=1.191922909389051, ranking=9.961297686073475
f1=0.29098396500106816, f2=0.15796543380307781, f3=1.0576916302859942, ranking=9.117250687106791
f1=0.24104308352686132, f2=0.7208433437117368, f3=0.8788251168412522, ranking=9.35588096290716
f1=1.1152112417580682, f2=0.08101936995241665, f3=0.46375838249493917, ranking=9.335029026453244
f1=0.4375790367965351, f2=0.6347074881695178, f3=0.9954327966069716, ranking=10.093383841024211
f1=0.8092340722313303, f2=0.58820722552